In [1]:
from scipy.sparse import csr_matrix
import pandas as pd
import implicit
import numpy as np
from scipy.sparse import hstack

/Users/vladimirkalajcidi/miniconda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd ..

/Users/vladimirkalajcidi/vk_recsys


In [5]:
user_features = pd.read_parquet('data/users_meta.parquet')
item_features = pd.read_parquet('data/items_meta.parquet')

In [6]:
item_features

,item_id,source_id,duration,embeddings
0,0,1869,13,"[0.07607838, 0.24312432, -0.07126159, 0.068710..."
1,1,1869,8,"[-0.2897247, 0.016220074, -0.14935663, 0.08754..."
2,2,1869,10,"[-0.08479501, 0.16359554, -0.12039807, 0.09242..."
3,3,1869,49,"[-0.061852332, 0.14139329, -0.00040801117, -0...."
4,4,1869,8,"[-0.12159933, -0.019279012, -0.13987888, 0.020..."
...,...,...,...,...
337722,337722,14899,28,"[0.093476385, -0.1326654, -0.3632401, -0.09967..."
337723,337723,14899,33,"[-0.06391368, 0.24055511, -0.30394536, 0.02770..."
337724,337724,10206,18,"[-0.06878064, 0.39626905, -0.06809371, -0.1031..."
337725,337725,19530,15,"[0.22456898, -0.2991999, -0.21905008, -0.12759..."


In [18]:
interactions = pd.read_parquet("data/train_interactions.parquet")

interactions['like'] = interactions['like'].astype('int32')    # Convert to signed integer
interactions['dislike'] = interactions['dislike'].astype('int32')  # Convert to signed integer

# Now create the 'weight' column
interactions['label'] = interactions['like'] - interactions['dislike']
interactions = interactions.merge(user_features, on='user_id', how='left')
interactions = interactions.merge(item_features, on='item_id', how='left')


interactions = interactions[['user_id', 'item_id', 'gender', \
                            'age', 'source_id', 'duration', 'label']]

new_mapping = {
    1: 2,      # Change to 2
    0: 1,    # Change to 1
    -1: 0    # Change to 0
}

# Use replace to update the interaction column
interactions['label'] = interactions['label'].replace(new_mapping)

X = interactions.drop(columns = ['label']).values
y = interactions['label'].values
users = interactions['user_id'].values

group_sizes = interactions.groupby('user_id').size().tolist()

'''items_meta = pd.read_parquet("data/items_meta.parquet")
users_meta = pd.read_parquet("data/users_meta.parquet")
n_items = items_meta["item_id"].max() + 1
n_users = users_meta["user_id"].max() + 1

data_weight = csr_matrix((interactions["weight"],
                    (interactions["user_id"].to_numpy(),
                     interactions["item_id"].to_numpy())),
                   shape=(n_users, n_items))

data_time = csr_matrix((interactions["timespent"],
                    (interactions["user_id"].to_numpy(),
                     interactions["item_id"].to_numpy())),
                   shape=(n_users, n_items))'''

'items_meta = pd.read_parquet("data/items_meta.parquet")\nusers_meta = pd.read_parquet("data/users_meta.parquet")\nn_items = items_meta["item_id"].max() + 1\nn_users = users_meta["user_id"].max() + 1\n\ndata_weight = csr_matrix((interactions["weight"],\n                    (interactions["user_id"].to_numpy(),\n                     interactions["item_id"].to_numpy())),\n                   shape=(n_users, n_items))\n\ndata_time = csr_matrix((interactions["timespent"],\n                    (interactions["user_id"].to_numpy(),\n                     interactions["item_id"].to_numpy())),\n                   shape=(n_users, n_items))'

In [13]:
new_mapping = {
    1: 2,      # Change to 2
    0: 1,    # Change to 1
    -1: 0    # Change to 0
}

# Use replace to update the interaction column
interactions['label'] = interactions['label'].replace(new_mapping)

X = interactions.drop(columns = ['label']).values
y = interactions['label'].values
users = interactions['user_id'].values

group_sizes = interactions.groupby('user_id').size().tolist()

In [ ]:
group_sizes

In [19]:
import lightgbm as lgb

# Create LightGBM dataset
lgb_data = lgb.Dataset(X, label=y, group=group_sizes)

In [20]:
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
}

ranker = lgb.train(params, lgb_data, num_boost_round=100)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.672699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 989
[LightGBM] [Info] Number of data points in the train set: 145667282, number of used features: 6


In [56]:
ranker_model = lgb.LGBMRanker(n_estimators = 20,
                         random_state = 33,
                         n_jobs = 8
                         )

study_cols = ['user_id', 'item_id', 'gender', \
                            'age', 'source_id', 'duration']

In [57]:
import gc
result_lst = []
result_lst.append(interactions)
gc.collect()

3466

In [58]:
result_lst

[           user_id  item_id  gender  age  source_id  duration  label
 0             3810   138979       1   36       4278        54      1
 1           101874   331160       2   52       2049         6      1
 2           150332    73709       1   24      16375        16      1
 3             4982   189745       1   40        166        25      1
 4           149601   289643       1   34       1459        23      1
 ...            ...      ...     ...  ...        ...       ...    ...
 145667277    10718    93558       1   36      17864        54      1
 145667278   119164    60206       2   24      13642        59      1
 145667279   171518    99323       1   38      13584        39      1
 145667280   109533    74203       2   20       9528        25      1
 145667281   129851   138079       2   23      17500        24      1
 
 [145667282 rows x 7 columns]]

In [59]:
result_df = pd.concat(result_lst).reset_index(drop=True)
result_df = result_df.sort_values('user_id').reset_index(drop=True)
group_pretrain = result_df.groupby('user_id').size().reset_index(name='cnt').cnt.values

In [60]:
result_df

,user_id,item_id,gender,age,source_id,duration,label
0,0,47731,2,35,5419,42,1
1,0,201247,2,35,14389,20,2
2,0,212815,2,35,13414,19,1
3,0,115821,2,35,6877,22,1
4,0,225255,2,35,15522,55,1
...,...,...,...,...,...,...,...
145667277,183403,50452,1,29,12980,43,1
145667278,183403,50178,1,29,12980,107,1
145667279,183403,266368,1,29,16039,32,1
145667280,183403,229142,1,29,16850,52,1


In [61]:
ranker_model.fit(result_df[study_cols], 
          result_df['label'], 
          group=group_pretrain
         )

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.684501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 989
[LightGBM] [Info] Number of data points in the train set: 145667282, number of used features: 6


LGBMRanker(n_estimators=20, n_jobs=8, random_state=33)

In [30]:
from tqdm import tqdm

In [62]:
import gc
from tqdm import tqdm
full_result_df_lst = []
for i in tqdm(range(10)):
    test_pairs = pd.read_csv('data/test_pairs.csv')

    test_pairs = test_pairs.merge(user_features, on='user_id', how='left')
    test_pairs = test_pairs.merge(item_features, on='item_id', how='left')


    test_pairs = test_pairs[['user_id', 'item_id', 'gender', 'age', 'source_id', 'duration']]


    test_pairs = test_pairs.sort_values('user_id').reset_index(drop=True)
    group_pretrain = test_pairs.groupby('user_id').size().reset_index(name='cnt').cnt.values
    test_pairs['predict'] = ranker_model.predict(test_pairs[['user_id', 'item_id', 'gender', 'age', 'source_id', 'duration']], group=group_pretrain)

100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [40]:
test_pairs = test_pairs[['user_id', 'item_id', 'predict']]

In [70]:
test_pairs = test_pairs.sort_values(['predict'], ascending = True).groupby('user_id')

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

In [71]:
result_df = pd.concat([test_pairs]).reset_index(drop=True)

TypeError: cannot concatenate object of type '<class 'pandas.core.groupby.generic.DataFrameGroupBy'>'; only Series and DataFrame objs are valid

In [72]:
test_pairs_final = pd.read_csv('data/test_pairs.csv')

test_pairs = test_pairs_final.merge(test_pairs[['user_id', 'item_id', 'predict']], 
                                   on=['user_id', 'item_id'], 
                                   how='left')

TypeError: Can only merge Series or DataFrame objects, a <class 'pandas.core.groupby.generic.DataFrameGroupBy'> was passed

In [65]:
test_pairs

,user_id,item_id,predict
0,1,7363,0.464024
1,1,73770,0.452454
2,1,75700,0.418140
3,1,81204,0.484344
4,1,110249,0.501709
...,...,...,...
1655115,183403,235097,0.528466
1655116,183403,267411,0.488367
1655117,183403,273646,0.495473
1655118,183403,288305,0.487833


In [ ]:
merged_df = random_order_df.merge(result_df[['user_id', 'item_id', 'predict']], 
                                   on=['user_id', 'item_id'], 
                                   how='left')

In [5]:
user_features = pd.read_parquet('data/users_meta.parquet')

user_feature_matrix = csr_matrix(user_features.drop(columns='user_id').values)

In [ ]:
item_features_data = pd.read_parquet('data/items_meta.parquet')

embeddings_array = np.array(item_features_data['embeddings'].tolist())

# Convert the embedding array to a DataFrame
embedding_df = pd.DataFrame(embeddings_array, columns=[f'embedding_{i}' for i in range(32)])

# Concatenate the DataFrame with the embeddings
item_features_data = pd.concat([item_features_data, embedding_df], axis=1)

item_features_data = item_features_data.drop(columns=['source_id', 'duration', 'embeddings'])

item_feature_matrix = csr_matrix(item_features_data.drop(columns='item_id').values) 

In [21]:
user_data_weight_combined = hstack([data_weight, user_feature_matrix])
user_data_time_combined = hstack([data_time, user_feature_matrix])

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=512,
                                             iterations=100,
                                             regularization=2.5,
                                             calculate_training_loss=True)
model.fit(user_data_weight_combined)

  1%|          | 1/100 [06:28<9:04:27, 329.98s/it, loss=6.07e-6]

In [21]:
test_pairs = pd.read_csv('data/test_pairs.csv')

test_pairs = test_pairs.merge(user_features, on='user_id', how='left')
test_pairs = test_pairs.merge(item_features, on='item_id', how='left')


test_pairs = test_pairs[['user_id', 'item_id', 'gender', \
                            'age', 'source_id', 'duration']]


In [24]:
# Example feature extraction function
def extract_features(user_id, item_id, gender, age, source_id, duration):
    feature_1 = user_id
    feature_2 = item_id
    # Convert categorical variables to numeric
    gender_encoded = gender
    # You may need one-hot encoding for source_id or more advanced encoding

    # Create features
    # Here you can include more complex feature engineering as needed
    feature_3 = gender_encoded  # Normalize age as a simple feature
    feature_4 = age  # Normalize duration
    feature_5 = source_id  # Use source ID as a feature directly
    feature_6 = duration  # Use encoded gender
    return [feature_1, feature_2, feature_3, feature_4, feature_5, feature_6]

# Extract features for each user-item pair
features = [
    extract_features(row['user_id'], 
                     row['item_id'], 
                     row['gender'], 
                     row['age'], 
                     row['source_id'], 
                     row['duration']) 
    for _, row in test_pairs.iterrows()
]

# Convert the list of features to a NumPy array
X_predict = np.array(features)

# Assuming 'ranker' is your trained LightGBM model
# Make predictions using the trained model
predictions = ranker.predict(X_predict)

# Add the predictions to the original DataFrame without sorting
test_pairs['predict'] = predictions

# Display the final DataFrame with predictions
print(test_pairs)

         user_id  item_id  gender  age  source_id  duration   predict
0              1     7363       2   35       5119         9 -0.058485
1              1    73770       2   35       2720         5  0.074801
2              1    75700       2   35      11601        32 -1.401590
3              1    81204       2   35       4296        10 -0.138425
4              1   110249       2   35      18812        38 -1.578304
...          ...      ...     ...  ...        ...       ...       ...
1655115   183403   235097       1   29      11668        46 -1.705747
1655116   183403   267411       1   29       1438        47 -1.677157
1655117   183403   273646       1   29       5158        27 -1.277589
1655118   183403   288305       1   29       4848        19 -1.009722
1655119   183403   322029       1   29       9247        54 -1.728839

[1655120 rows x 7 columns]


In [25]:
test_pairs = test_pairs.drop(columns=['gender', \
                            'age', 'source_id', 'duration'])

In [66]:
test_pairs.to_csv('sample_submission.csv', index=False)

In [26]:
test_pairs

,user_id,item_id,predict
0,1,7363,-0.058485
1,1,73770,0.074801
2,1,75700,-1.401590
3,1,81204,-0.138425
4,1,110249,-1.578304
...,...,...,...
1655115,183403,235097,-1.705747
1655116,183403,267411,-1.677157
1655117,183403,273646,-1.277589
1655118,183403,288305,-1.009722


In [23]:
X_predict

array([[   35,     9,     2,  5119],
       [   35,     5,     2,  2720],
       [   35,    32,     2, 11601],
       ...,
       [   29,    27,     1,  5158],
       [   29,    19,     1,  4848],
       [   29,    54,     1,  9247]])

In [9]:
als_predict_weight = (model.user_factors[test_pairs['user_id']] *
               model.item_factors[test_pairs['item_id']]).sum(axis=1)

In [10]:
model = implicit.als.AlternatingLeastSquares(factors=50,
                                             iterations=20,
                                             regularization=1,
                                             alpha=100,
                                             calculate_training_loss=True)
model.fit(user_data_weight_combined)

100%|██████████| 20/20 [25:28<00:00, 76.44s/it, loss=0.00372]


In [11]:
als_predict_time = (model.user_factors[test_pairs['user_id']] *
               model.item_factors[test_pairs['item_id']]).sum(axis=1)

In [18]:
import sklearn

In [19]:
als_cosine_time = sklearn.metrics.pairwise.cosine_similarity(model.user_factors[test_pairs['user_id']],
                                                            model.item_factors[test_pairs['item_id']])

AttributeError: module 'sklearn' has no attribute 'metrics'

In [15]:
min(als_predict_time)

-0.8893903

In [27]:
# Add the predictions as a new column
test_pairs['predict'] = als_predict_time

# Save the DataFrame to a CSV file
test_pairs.to_csv('sample_submission.csv', index=False)

NameError: name 'als_predict_time' is not defined